In [1]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 40.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [47]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor

from rdkit import Chem
from rdkit.Chem import Descriptors

In [48]:
df = pd.read_csv('/content/drive/MyDrive/Thesis_Implementation/Dataset/test/forranking.csv')

In [58]:
smiles = df['SMILES']
mols = [Chem.MolFromSmiles(x) for x in smiles]
df['MolWeight'] = [Descriptors.MolWt(x) for x in mols]
df['LogP'] = [Descriptors.MolLogP(x) for x in mols]
#df['TPSA'] = [Descriptors.TPSA(x) for x in mols]  #Topological Polar Surface Area
df['HBD'] = [Descriptors.NumHDonors(x) for x in mols]
df['HBA'] = [Descriptors.NumHAcceptors(x) for x in mols]
#df['MolecularFormula'] = [Chem.MolToFormula(x) for x in mols]
#df['NumRings'] = [Chem.GetSSSR(x) for x in mols]

In [59]:
train_data = df.sample(frac=0.8, random_state=0)
test_data = df.drop(train_data.index)

In [60]:
X_train = train_data['MolWeight', 'LogP', 'HBD', 'HBA']
y_train = train_data['Affinity']

model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

KeyError: ignored

In [23]:
X_test = test_data[['MolWeight', 'LogP', 'HBD', 'HBA']]
y_test = test_data['Affinity']

y_pred = model.predict(X_test)

In [24]:
error = np.abs(y_pred - y_test)
mean_error = np.mean(error)
print("Mean absolute error:", mean_error)

Mean absolute error: 0.4264094727012502


In [25]:
df['PredictedAffinity'] = model.predict(df[['MolWeight', 'LogP', 'HBD', 'HBA', 'NumRings']])
#df = df.sort_values(by='PredictedAffinity', ascending=False)

df = df.sort_values(by=['MolWeight', 'PredictedAffinity', 'HBD', 'HBA', 'NumRings'], ascending=[True, True, False, False, False])

In [27]:
print(df.head(10))

                                              SMILES  Affinity  MolWeight  \
1  C(NC(=O)C1C2OC(CN(C(NO)=O)C2)O1)c1ccc(-c2ccccc...  8.267059    383.404   
2  c1(CNC(C2OC3OC2CN(C(NO)=O)C3)=O)ccc(-c2ccccc2)cc1  7.110216    383.404   
4  c1cc(CNC(=O)C2C3OC(CN(C(=O)NO)C3)O2)ccc1-c1ccccc1  9.155649    383.404   
5  C12CN(C(NO)=O)CC(O1)C(C(=O)NCc1ccc(-c3ccccc3)c...  7.597659    383.404   
6    O1C2CN(C(NO)=O)CC1OC2C(=O)NCc1ccc(-c2ccccc2)cc1  7.448720    383.404   
0  ONC(N1CC2OC(C(=O)NCc3ccc(-c4ccccc4)cc3)C(C1)O2)=O  6.962898    383.404   
3  ONC(N1CC2OC(C(NCc3ccc(-c4ccccc4)cc3)=O)C(C1)O2)=O  6.818207    383.404   
7  N(C(=O)N1CC2OC(C(NCc3ccc(-c4ccccc4)cc3)=O)C(O2...  6.552645    383.404   
8    c1cc(CNC(=O)C2OC3OC2CN(C(=O)NO)C3)ccc1-c1ccccc1  9.241339    383.404   
9  C12OC(C(NCc3ccc(-c4ccccc4)cc3)=O)C(O1)CN(C(NO)...  6.931837    383.404   

     LogP  PredictedAffinity  
1  1.4943           7.706688  
2  1.4943           7.706688  
4  1.4943           7.706688  
5  1.4943           7.706688